# Time Series of YCSD Covid Case Metric

This notebook examines the time series of the York County COVID Cases ostensibly used by YCSD to make decisions about school reopenings.

* YCSD presents there metrics on https://ycsd.yorkcountyschools.org/domain/1313
* VDH represents the this number for the localities on https://www.vdh.virginia.gov/coronavirus/coronavirus/covid-19-in-virginia-locality/ and on https://www.vdh.virginia.gov/coronavirus/key-measures/pandemic-metrics/school-metrics/ under the localities tab
* VDH shares the data at https://data.virginia.gov/Government/VDH-COVID-19-PublicUseDataset-Cases/bre9-aqqr
* I'm sharing This notebook in Github at https://github.com/drf5n/YCSD_covid_metrics

-- David Forrest


In [1]:
%matplotlib widget
import os,sys,io, time
import pandas as pd
#import numpy as np, matplotlib as mpl, matplotlib.pyplot as plt

import bokeh.plotting
import bokeh.io
import bokeh.models
from bokeh.io import output_notebook
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
def file_age(filepath):
    return time.time() - os.path.getmtime(filepath)


In [3]:
# get the Virginia COVID Case data from https://data.virginia.gov/Government/VDH-COVID-19-PublicUseDataset-Cases/bre9-aqqr

df_name = "VA_vdh_casedata.csv"
if 1 or file_age(df_name) > 86400:
    !wget -O $df_name 'https://data.virginia.gov/api/views/bre9-aqqr/rows.csv?accessType=DOWNLOAD'

--2020-11-23 10:35:14--  https://data.virginia.gov/api/views/bre9-aqqr/rows.csv?accessType=DOWNLOAD
Resolving data.virginia.gov (data.virginia.gov)... 52.206.68.26, 52.206.140.199, 52.206.140.205
Connecting to data.virginia.gov (data.virginia.gov)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘VA_vdh_casedata.csv’

VA_vdh_casedata.csv     [   <=>              ]   1.55M  2.67MB/s    in 0.6s    

2020-11-23 10:35:15 (2.67 MB/s) - ‘VA_vdh_casedata.csv’ saved [1630264]



In [4]:
df=pd.read_csv(df_name)
df["date"] = pd.to_datetime(df['Report Date'])

df.tail()

,Report Date,FIPS,Locality,VDH Health District,Total Cases,Hospitalizations,Deaths,date
33511,11/23/2020,51800,Suffolk,Western Tidewater,2631,158,80,2020-11-23
33512,11/23/2020,51810,Virginia Beach,Virginia Beach,10168,507,112,2020-11-23
33513,11/23/2020,51820,Waynesboro,Central Shenandoah,560,17,5,2020-11-23
33514,11/23/2020,51830,Williamsburg,Peninsula,283,17,8,2020-11-23
33515,11/23/2020,51840,Winchester,Lord Fairfax,890,52,4,2020-11-23


In [5]:

df = df.sort_values(by=['Locality', 'VDH Health District', 'date'])
display(df.head())

df['TC_diff']= df.groupby('Locality')['Total Cases'].diff().fillna(0)
df['TC_sum14']= df.groupby('Locality')['Total Cases'].diff(14).fillna(0)

display(df.tail())



,Report Date,FIPS,Locality,VDH Health District,Total Cases,Hospitalizations,Deaths,date
0,03/17/2020,51001,Accomack,Eastern Shore,0,0,0,2020-03-17
133,03/18/2020,51001,Accomack,Eastern Shore,0,0,0,2020-03-18
266,03/19/2020,51001,Accomack,Eastern Shore,0,0,0,2020-03-19
399,03/20/2020,51001,Accomack,Eastern Shore,1,0,0,2020-03-20
532,03/21/2020,51001,Accomack,Eastern Shore,1,0,0,2020-03-21


,Report Date,FIPS,Locality,VDH Health District,Total Cases,Hospitalizations,Deaths,date,TC_diff,TC_sum14
32945,11/19/2020,51199,York,Peninsula,783,26,9,2020-11-19,8.0,113.0
33078,11/20/2020,51199,York,Peninsula,806,26,9,2020-11-20,23.0,120.0
33211,11/21/2020,51199,York,Peninsula,817,27,9,2020-11-21,11.0,120.0
33344,11/22/2020,51199,York,Peninsula,825,27,9,2020-11-22,8.0,118.0
33477,11/23/2020,51199,York,Peninsula,848,27,9,2020-11-23,23.0,138.0


In [15]:
# subset for York and normalize per capita
dfy = df[df['Locality']=='York'].copy()
dfy['per100k_14daysum']=dfy['TC_sum14']*100000/67782  

In [16]:
dfy

,Report Date,FIPS,Locality,VDH Health District,Total Cases,Hospitalizations,Deaths,date,TC_diff,TC_sum14,per100k_14daysum
46,03/17/2020,51095,James City,Peninsula,12,6,1,2020-03-17,0.0,0.0,0.000000
179,03/18/2020,51095,James City,Peninsula,13,4,1,2020-03-18,1.0,0.0,0.000000
312,03/19/2020,51095,James City,Peninsula,14,4,1,2020-03-19,1.0,0.0,0.000000
445,03/20/2020,51095,James City,Peninsula,19,5,1,2020-03-20,5.0,0.0,0.000000
578,03/21/2020,51095,James City,Peninsula,20,5,1,2020-03-21,1.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
32897,11/19/2020,51095,James City,Peninsula,1082,71,22,2020-11-19,5.0,97.0,143.105839
33030,11/20/2020,51095,James City,Peninsula,1090,71,22,2020-11-20,8.0,100.0,147.531793
33163,11/21/2020,51095,James City,Peninsula,1111,71,22,2020-11-21,21.0,114.0,168.186244
33296,11/22/2020,51095,James City,Peninsula,1121,71,22,2020-11-22,10.0,115.0,169.661562


In [8]:
ph = dfy.plot(y='per100k_14daysum',x='date',title="York County Number of new cases per 100,000 persons \nwithin the last 14 days")

ph

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:title={'center':'York County Number of new cases per 100,000 persons \nwithin the last 14 days'}, xlabel='date'>

In [9]:
ph = dfy.plot(y='TC_diff',x='date',title="York County Cases, 14 day sum, per 100K")
ph

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:title={'center':'York County Cases, 14 day sum, per 100K'}, xlabel='date'>

In [10]:
TOOLTIPS = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
    ("radius", "@radius"),
]


#p=bokeh.plotting.figure( tooltips=TOOLTIPS, x_axis_type='datetime')
p=bokeh.plotting.figure( x_axis_type='datetime',y_range=(0,250),title="York County Number of new cases per 100,000 persons within the last 14 days")
p.add_tools(bokeh.models.HoverTool())

#hover = p.select(dict(type=bokeh.models.HoverTool))


#hover(tooltips=TOOLTIPS,
#)

p.add_layout(bokeh.models.BoxAnnotation(bottom=0,top=5, fill_alpha=0.4, fill_color='olive'))
p.add_layout(bokeh.models.BoxAnnotation(bottom=5,top=20, fill_alpha=0.4, fill_color='green'))
p.add_layout(bokeh.models.BoxAnnotation(bottom=20,top=50, fill_alpha=0.4, fill_color='yellow'))
p.add_layout(bokeh.models.BoxAnnotation(bottom=50,top=200, fill_alpha=0.4, fill_color='orange'))
p.add_layout(bokeh.models.BoxAnnotation(bottom=200, fill_alpha=0.4, fill_color='red'))



p.line(dfy['date'],dfy['per100k_14daysum'])
#p.title()

#?p.line

GlyphRenderer(id='1045', ...)

In [11]:
bokeh.plotting.show(p)

In [12]:
bokeh.plotting.output_file('YorkCountyCovidMetric_plot.html', mode='inline')
bokeh.plotting.save(p)

# needs geckodriver  -- have it in conda env py3plot
bokeh.io.export_png(p, filename="YorkCountyCovidMetric_plot.png")



'/Users/drf/2020/Home/School/YCSD_covid_metrics/YorkCountyCovidMetric_plot.png'